# Nifty Expiry Predictor - Data Exploration

This notebook demonstrates data exploration and analysis capabilities.

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data.collectors.nse_scraper import NSEDataCollector
from features.greeks_calculator import GreeksCalculator
from features.gex_calculator import GammaExposureCalculator
from features.max_pain import MaxPainCalculator
from features.oi_analysis import OIAnalyzer

%matplotlib inline
sns.set_style('darkgrid')

## 1. Fetch Option Chain Data

In [ ]:
# Initialize NSE data collector
nse = NSEDataCollector()

# Fetch option chain
option_chain = nse.get_option_chain('NIFTY')
spot = option_chain.attrs['underlying']

print(f"Spot Price: {spot}")
print(f"Number of Strikes: {len(option_chain)}")
option_chain.head()

## 2. Calculate GEX

In [ ]:
# Calculate GEX
gex_calc = GammaExposureCalculator()
gex_df = gex_calc.calculate_chain_gex(option_chain, spot, days_to_expiry=1)

# Find GEX levels
gex_levels = gex_calc.find_gex_levels(gex_df, spot)

print(f"GEX Regime: {gex_levels['gex_regime']}")
print(f"Trading Bias: {gex_levels['trading_bias']}")
print(f"Total GEX: {gex_levels['total_gex']:,.0f}")

## 3. Visualize GEX Profile

In [ ]:
# Plot GEX profile
plt.figure(figsize=(12, 6))
plt.bar(gex_df['strike'], gex_df['net_gex'], 
        color=gex_df['net_gex'].apply(lambda x: 'green' if x > 0 else 'red'),
        alpha=0.7)
plt.axvline(x=spot, color='blue', linestyle='--', label=f'Spot: {spot}')
plt.xlabel('Strike Price')
plt.ylabel('Net GEX')
plt.title('Gamma Exposure Profile')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Calculate Max Pain

In [ ]:
# Calculate Max Pain
mp_calc = MaxPainCalculator()
max_pain, pain_df = mp_calc.calculate_max_pain(option_chain)

print(f"Max Pain: {max_pain}")
print(f"Distance from Spot: {spot - max_pain}")

## 5. Visualize Max Pain Curve

In [ ]:
# Plot Max Pain curve
plt.figure(figsize=(12, 6))
plt.plot(pain_df['strike'], pain_df['total_pain'], 'r-', linewidth=2)
plt.axvline(x=max_pain, color='green', linestyle='--', label=f'Max Pain: {max_pain}')
plt.axvline(x=spot, color='blue', linestyle='--', label=f'Spot: {spot}')
plt.xlabel('Strike Price')
plt.ylabel('Total Pain')
plt.title('Max Pain Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. OI Analysis

In [ ]:
# Analyze OI
oi_analyzer = OIAnalyzer()

# Calculate PCR
pcr = oi_analyzer.calculate_pcr(option_chain, method='oi')
sentiment = oi_analyzer.interpret_pcr(pcr)

print(f"Put-Call Ratio: {pcr:.2f}")
print(f"Market Sentiment: {sentiment}")

# Find OI walls
walls = oi_analyzer.find_call_put_walls(option_chain, num_walls=5)
print("\nCall Walls:")
for wall in walls['call_walls']:
    print(f"  Strike: {wall['strike']}, OI: {wall['call_oi']:,.0f}")

print("\nPut Walls:")
for wall in walls['put_walls']:
    print(f"  Strike: {wall['strike']}, OI: {wall['put_oi']:,.0f}")